<a href="https://colab.research.google.com/github/Ansebi/causal_inference/blob/kls/granger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install bamt

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

In [ ]:
import bamt.networks as networks
import bamt.preprocessors as pp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import train_test_split

from pgmpy.estimators import K2Score, BicScore, BDeuScore, StructureScore

# Data

In [ ]:
def normalize(
    array: np.array,
    min_: float = None,
    max_: float = None
):
  if len(np.unique(array)) == 1:
    value = np.unique(array)[0]
    norm_value = 0.5
    if max_ != min_:
      norm_value = (value - min_) / (max_ - min_)
    return np.ones_like(array) * norm_value
  if min_ is None:
    min_ = array.min()
  if max_ is None:
    max_ = array.max()
  return (array - min_) / (max_ - min_)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


N_GARBAGE = 3
LAG = 5
# past = np.array([1, 7, 9, 2, 1])
# past = np.array([1, 7, 9, 2, 1] * 10)
past = np.random.randint(0, 9, 1000)
future = np.zeros_like(past).astype(float)
for i in range(len(past)):
    if i - LAG >= 0:
        value = past[i-LAG] * 2 + (np.random.random() - 1) * 1e-1
    else:
        value = np.random.randint(0, 9)
    future[i] = value
garbage = {f'garbage_{i}': np.random.randint(0, 9, len(past)) for i in range(N_GARBAGE)}

df = pd.DataFrame(
    {
        'past': past,
        'future': future,
        **garbage
    }
)

df = df.apply(normalize)
df.head(15)

,past,future,garbage_0,garbage_1,garbage_2
0,1.000,0.068309,0.875,0.875,0.625
1,0.625,0.503201,0.500,0.125,0.750
2,0.750,0.503201,0.250,0.250,0.000
3,1.000,0.192564,0.250,0.250,0.500
4,0.625,0.130437,0.125,0.750,0.375
5,0.500,0.995480,0.625,0.375,0.500
6,0.750,0.622151,0.500,0.375,0.250
7,0.125,0.746337,0.375,0.000,0.625
8,0.375,0.996430,1.000,0.125,0.625
9,0.500,0.623359,0.875,0.500,0.375


# grangercausalitytests

In [ ]:
pip install statsmodels --upgrade --q

In [ ]:
import IPython

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag = 10
test = 'ssr_chi2test'

def grangers_causation_matrix(
    data, variables, test='ssr_chi2test', verbose=False
):
    results = {}
    len_ = len(variables)
    granger_matrix = pd.DataFrame(
        np.zeros((len_, len_)),
        columns=variables,
        index=variables
      )
    for column in granger_matrix.columns:
      results[column] = {}
      for row in granger_matrix.index:
        result = grangercausalitytests(
          data[[row, column]], maxlag=maxlag, verbose=False
        )
        results[column][row] = result
        IPython.display.clear_output()
        p_values = [result[i+1][0][test][1] for i in range(maxlag)]
        if verbose:
          print(f'Y = {row}, X = {column}, P Values = {p_values}')
        min_p_value = np.min(p_values)
        granger_matrix.loc[row, column] = min_p_value
    return granger_matrix, results

grangers_matrix, results = grangers_causation_matrix(df, variables=df.columns)
grangers_matrix

,past,future,garbage_0,garbage_1,garbage_2
past,1.000000,0.309235,0.052278,0.483616,0.064860
future,0.000000,1.000000,0.002352,0.486013,0.096396
garbage_0,0.018470,0.003068,1.000000,0.081643,0.389230
garbage_1,0.190997,0.115503,0.551499,1.000000,0.138624
garbage_2,0.348213,0.648932,0.058119,0.239778,1.000000


- для целевой переменной прокрасились (p-value < 0.05) две фичи - past и garbage_0

# Bamt

In [ ]:
encoder = preprocessing.LabelEncoder()
discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

p = pp.Preprocessor([('encoder', encoder), ('discretizer', discretizer)])
discretized_data, est = p.apply(df)

2024-03-25 16:00:41,956 | INFO     | preprocessors.py-scan-0107 | No one column is discrete


In [ ]:
bn = networks.ContinuousBN(use_mixture=True) # init BN

info = p.info
bn.add_nodes(info)

In [ ]:
bn.add_edges(discretized_data,  scoring_function=('K2', K2Score))

  0%|          | 0/1000000 [00:00<?, ?it/s]

2024-03-25 16:00:52,197 | ERROR    | builders_base.py-get_family-0082 | Edges list is None


In [ ]:
bn.get_info()

,name,node_type,data_type,parents,parents_types
0,past,MixtureGaussian,cont,[],[]
1,future,MixtureGaussian,cont,[],[]
2,garbage_0,MixtureGaussian,cont,[],[]
3,garbage_1,MixtureGaussian,cont,[],[]
4,garbage_2,MixtureGaussian,cont,[],[]


In [ ]:
bn.calculate_weights(discretized_data)
bn.weights

2024-03-25 16:01:01,255 | ERROR    | base.py-calculate_weights-0219 | Bayesian Network hasn't fitted yet. Please add edges with add_edges() method


{}

- bamt не увидел ни одной зависимости в данных с временным сдвигом

# Reverse experiment

In [ ]:
N_GARBAGE = 3
magnitude = np.random.randint(0, 9, 1000)

depends_on_magnitude = magnitude * 10 + np.random.rand(len(magnitude)) / 10
switch = np.ones_like(magnitude)
garbage = {f'garbage_{i}': np.random.random(len(magnitude)) for i in range(N_GARBAGE)}
outcome = depends_on_magnitude + 1 + np.random.rand(len(magnitude)) / 10
df = pd.DataFrame(
    {
        'magnitude': magnitude,
        'depends_on_magnitude': depends_on_magnitude,
        'switch': switch,
        **garbage,
        'outcome': outcome
    }
)

df = df.apply(normalize)
df.head(15)

,magnitude,depends_on_magnitude,switch,garbage_0,garbage_1,garbage_2,outcome
0,0.625,0.624782,0.5,0.222863,0.074773,0.411264,0.624654
1,0.750,0.749719,0.5,0.421263,0.707017,0.405030,0.749972
2,0.000,0.001144,0.5,0.661679,0.970990,0.329413,0.001219
3,0.625,0.624810,0.5,0.977010,0.760968,0.534183,0.625110
4,0.875,0.874162,0.5,0.000000,0.463486,0.567203,0.873982
5,0.625,0.624554,0.5,0.273688,0.344361,0.115382,0.624156
6,0.000,0.000138,0.5,0.956152,0.068646,0.690534,0.001287
7,0.500,0.500159,0.5,0.421042,0.791972,0.558933,0.499772
8,0.875,0.874097,0.5,0.624685,0.945188,0.733568,0.874028
9,0.750,0.749091,0.5,0.361745,0.915769,0.832418,0.748451


In [ ]:
grangers_matrix, results = grangers_causation_matrix(df.drop(columns=['switch']), variables=df.drop(columns=['switch']).columns)
grangers_matrix

,magnitude,depends_on_magnitude,garbage_0,garbage_1,garbage_2,outcome
magnitude,1.000000,0.061068,0.156529,0.149281,0.414410,0.153826
depends_on_magnitude,0.061053,1.000000,0.156190,0.149166,0.414598,0.084545
garbage_0,0.031609,0.031591,1.000000,0.115995,0.650748,0.031538
garbage_1,0.229394,0.229237,0.012711,1.000000,0.050691,0.229785
garbage_2,0.165014,0.164818,0.517223,0.172009,1.000000,0.165214
outcome,0.153099,0.084572,0.156175,0.148797,0.413385,1.000000


- нельзя использвать константные колонки
- нельзя задать нулевой лаг